In [1]:
include("Tools.jl");
include("GaugeFixing.jl")
include("KrylovTechnical.jl")
using DataFrames, CSV;

Note that this notebook does not evaluate anything. It only loads, displays, and saves in a human friendly format the precomputed results.

In [2]:
#CHOOSE YOUR VALUES HERE####
############################
chi = 30                  ##
gilt_eps = 4e-6           ##
############################
############################

gilt_pars = Dict(
    "gilt_eps" => gilt_eps,
    "cg_chis" => collect(1:chi),
    "cg_eps" => 1e-10,
    "verbosity" => 0,
    "rotate" => false
)
gilt_pars_r = Dict(
    "gilt_eps" => gilt_eps,
    "cg_chis" => collect(1:chi),
    "cg_eps" => 1e-10,
    "verbosity" => 0,
    "rotate" => true
);


## Critical temperature with and without rotation (10 digits ):

In [3]:
id = gilt_pars_identifier(gilt_pars)
id_r = gilt_pars_identifier(gilt_pars_r)

crit_pars = deserialize("critical_temperatures/" * id * "__tol=1.0e-10.data")
crit_pars_r = deserialize("critical_temperatures/" * id_r * "__tol=1.0e-10.data")

println("             ", "relT lower bound   |  relT higher bound  | relT mean value")
println("no rotation  ", crit_pars[1], "  |  ", crit_pars[2], "  | ", (crit_pars[1] + crit_pars[2]) / 2)
println("   rotation  ", crit_pars_r[1], " |  ", crit_pars_r[2], " | ", (crit_pars_r[1] + crit_pars_r[2]) / 2)

             relT lower bound   |  relT higher bound  | relT mean value
no rotation  1.0000112570822237  |  1.0000112571567294  | 1.0000112571194766
   rotation  1.0000091544538736 |  1.0000091545283796 | 1.0000091544911265


# Spectra of the jacobian at the approximate fixed point (no rotation)

In [4]:
id_no_rotation_tt = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=true_freeze_R=true_eigensystem.data"
id_no_rotation_tf = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=true_freeze_R=false_eigensystem.data"
id_no_rotation_ft = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=false_freeze_R=true_eigensystem.data"
id_no_rotation_ff = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=false_freeze_R=false_eigensystem.data"


spectrum_Z2T_FT = deserialize(id_no_rotation_tt)["eigensystem"][1]
spectrum_Z2T_FF = deserialize(id_no_rotation_tf)["eigensystem"][1]
spectrum_Z2F_FT = deserialize(id_no_rotation_ft)["eigensystem"][1]
spectrum_Z2F_FF = deserialize(id_no_rotation_ff)["eigensystem"][1];

specs = DataFrame()
specs[:, "Z2 odd sector included; R freezing: true"] = spectrum_Z2T_FT[1:30]
specs[:, "Z2 odd sector included; R freezing: false"] = spectrum_Z2T_FF[1:30]
specs[:, "Z2 odd sector excluded; R freezing: true"] = spectrum_Z2F_FT[1:30]
specs[:, "Z2 odd sector excluded; R freezing: false"] = spectrum_Z2F_FF[1:30]
specs[:, "Z2 odd sector included; Ising CFT prediction"] = 2 .^ (2 .- exact_spectrum(31)[2:end])
specs[:, "Z2 odd sector excluded; Ising CFT prediction"] = 2 .^ (2 .- exact_spectrum_even(31)[2:end])

CSV.write("export/spectra_chi=$chi.csv", specs)
specs

Row,Z2 odd sector included; R freezing: true,Z2 odd sector included; R freezing: false,Z2 odd sector excluded; R freezing: true,Z2 odd sector excluded; R freezing: false,Z2 odd sector included; Ising CFT prediction,Z2 odd sector excluded; Ising CFT prediction
,Complex…,Complex…,Complex…,Complex…,Float64,Float64
1,4.63462+0.0im,3.66778+0.0im,2.00874+0.0im,1.99953+0.0im,3.66802,2.0
2,4.01798+0.0im,1.99953+0.0im,1.00103+0.0im,1.0031+0.0im,2.0,1.0
3,2.00874+0.0im,1.52863+0.0im,0.999168+0.0im,0.99775+0.0im,1.83401,1.0
4,1.55435+0.0im,1.52666+0.0im,0.806472+0.0409706im,0.652633+0.0im,1.83401,1.0
5,1.49706+0.0im,1.00311+0.0im,0.806472-0.0409706im,0.585554+0.0470158im,1.0,1.0
6,-1.23771+0.0im,0.997752+0.0im,0.662525+0.0im,0.585554-0.0470158im,1.0,0.5
7,1.08866+0.0im,0.880788+0.0im,0.558412+0.0im,0.441163+0.160744im,1.0,0.5
8,1.00103+0.0im,0.652281+0.0im,0.525541+0.0480959im,0.441163-0.160744im,1.0,0.5
9,0.999168+0.0im,0.586155+0.0398435im,0.525541-0.0480959im,0.378187+0.14452im,0.917004,0.5


## Spectrum of the jacobian after newton for Gilt with rotation

In [31]:
newton_result = deserialize("newton/" * gilt_pars_identifier(gilt_pars_r) * "__newton_result.data")
specs = DataFrame()
specs[:, "chi=$chi"] = newton_result["eigensystem"][1]

CSV.write("export/spectrum_chi=$(chi)_after_newton.csv", specs)
specs


Row,chi=10
,Complex…
1,2.00745+0.0im
2,-0.940972+0.0im
3,-0.936125+0.0im
4,0.444238+0.0im
5,-0.362702+0.0im
6,-2.78652e-7+0.318294im
7,-2.78652e-7-0.318294im
8,0.0627632+0.295995im
9,0.0627632-0.295995im
